In [1]:
#===========================================================================================
# ElasticSearch 텍스트 임베딩 테스트 예제
# -여기서는 elasticsearch 7.17.3 때를 기준으로 설명함.
# -** 따라서 elasticsearch python 모듈도 7.17.3 을 설치해야 함
#
# elasticsearch 모듈 8.x 부터는 구문의 많이 변경되었음.
# 예 : index 생성:  body로 모든 변수들를 지정하는 데시, 명시적으로 모든 변수들을 최상으로 지정해 줘야함.
# => 참고: https://towardsdatascience.com/important-syntax-updates-of-elasticsearch-8-in-python-4423c5938b17   
#
# =>ElasticSearch 7.3.0 버전부터는 cosine similarity 검색을 지원한다.
# => 데이터로 고차원벡터를 집어넣고, 벡터형식의 데이터를 쿼리(검색어)로 하여 
# 코사인 유사도를 측정하여 가장 유사한 데이터를 찾는다.
# => 여기서는 ElasticSearch와 S-BERT를 이용함
# => ElasticSearch에 index 파일은 index_1.json /데이터 파일은 KorQuAD_v1.0_train_convert.json 참조
#
# => 참고자료 : https://skagh.tistory.com/32
#
# => 참고로 여기서는 title_vector 만 구함, paragrapha_vector는 cpu에서는 엄청 오래 걸려서 주석처리하였음

#===========================================================================================

# sentenceTransformers 라이브러리 설치
#!pip install -U sentence-transformers

# elasticsearch 서버 접속 모듈 설치
# !pip install elasticsearch==7.17

# 한국어 문장 분리기(kss) 설치
#!pip install kss

In [2]:
import torch
import kss, numpy
import json
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from tqdm.notebook import tqdm

from elasticsearch import Elasticsearch
from elasticsearch import helpers

# elastic 서버 접속 테스트
#es = Elasticsearch("https://192.168.0.91:9200/", verify_certs=False)
#es = Elasticsearch("http://192.168.0.130:9200/")
#es.info()

device = torch.device('cpu')

# onnxmodel 일때는 True로 해줌.
onnxmodel=True

# es 인덱스 관련 변수들 정의
INDEX_NAME = 'korquadonnx'       #인덱스 명
INDEX_FILE = './data/index.json' #인덱스 구성 파일

# es 접속 서버 
ES_SERVER_IP_PORT= "http://192.168.0.27:9200/"

In [ ]:
'''
#===============================================================================================
# 1. sbert 모델 로딩
#===============================================================================================
from sentence_transformers import SentenceTransformer, util

# s-bert 모델 
sbert_model_path = '../../sentencebert_v1.0'

# cpu 모델로 실행할때는 device=cpu, 기본은 GPU임
embedder = SentenceTransformer(sbert_model_path, device=device)

text = '나는 오늘 밥을 먹는다.'
vectors = embedder.encode(text, convert_to_tensor=True)
vector_list = [vector.numpy().tolist() for vector in vectors]

print(f'vector_len:{len(vector_list)}')
'''

In [3]:
#===============================================================================================
# 1. ONNX 모델 로딩
#===============================================================================================

from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForFeatureExtraction

model_path='../../sbertonnx'
vocab_path='../../sbertonnx'

tokenizer=AutoTokenizer.from_pretrained(vocab_path)
model=ORTModelForFeatureExtraction.from_pretrained(model_path)
print(model)

# 테스트 
text = '나는 오늘 밥을 먹는다.'
input_token=tokenizer(text, truncation=True, padding=True, max_length=256, return_tensors="pt")
#print(input_token)

outputs = model(**input_token)
last_hidden_state = outputs.last_hidden_state
vectors = []
for hidden in last_hidden_state:
    # 평균값 구함
    mean_hidden = torch.mean(hidden, dim=0)
    vectors.append(mean_hidden)
    
print(len(vectors[0]))
#print(vectors)
        

768


In [4]:
#===============================================================================================
# 2. elasticsearch 접속
#===============================================================================================
es = Elasticsearch(ES_SERVER_IP_PORT)
print(es.info)

<bound method Elasticsearch.info of <Elasticsearch([{'host': '192.168.0.130', 'port': 9200}])>>


In [5]:
#===============================================================================================
# 3. Indices(index) 생성
#===============================================================================================

# index 삭제 후 재생성
def index_create(INDEX_NAME, INDEX_FILE):
    # 인덱스 삭제
    es.indices.delete(index=INDEX_NAME, ignore=[404])
    
    count = 0
    # 인덱스 생성
    with open(INDEX_FILE) as index_file:
        source = index_file.read().strip()
      
        count += 1
        print(f'{count}:{source}')
      
        es.indices.create(index=INDEX_NAME, body=source)
       
# korquadonnx 인덱스 생성    
index_create(INDEX_NAME, INDEX_FILE)

C:\Users\bong9\anaconda3\envs\pytorch\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


1:{
  "settings": {
    "number_of_shards": 2,
    "number_of_replicas": 1
  },
   "mappings": {
    "dynamic": "true",
    "_source": {
      "enabled": "true"
    },
    "properties": {
      "title": {
        "type": "text"
      },
	  "paragraph": {
        "type": "text"
      },
      "title_vector": {
        "type": "dense_vector",
        "dims": 768
      },
	  "paragraph_vector": {
        "type": "dense_vector",
        "dims": 768
      }
    }
  }
}


In [6]:
def index_batch(docs):
   
    titles = [doc["title"] for doc in docs]
    title_vectors = embed_text(titles)
    
    # 문장이 길면 분할해서 embedding을 구해야 하는데, 여기서는 분할하지 않고 embedding을 구함
    paragraphs = [doc["paragraph"] for doc in docs]
    paragraph_vectors = embed_text(paragraphs)
    
    # * cpu로 문단은 임베딩하는데 너무 오래 걸리므로 주석처리함
    #paragraph_vectors = [paragraph_index(doc["paragraph"]) for doc in tqdm(docs)]
    requests = []
    
    for i, doc in enumerate(tqdm(docs)):
        
        #request["title"] = doc["title"]           # 이렇게 등록은 안됨
        #request["paragraph"] = doc["paragraph"]
        
        request = doc
        
        request["_op_type"] = "index"
        request["_index"] = INDEX_NAME
        
        request["title_vector"] = title_vectors[i]
        request["paragraph_vector"] = paragraph_vectors[i]
        #request["paragraph_vector"] = paragraph_vectors[i]  # * cpu로 문단은 임베딩하는데 너무 오래 걸리므로 주석처리함
        
        requests.append(request)
        
    # batch 단위로 한꺼번에 es에 데이터 insert 시킴     
    bulk(es, requests)
    
# embedding 모델에서 vector를 구함    
def embed_text(input):
    #===================================================================
    # onnx 모델일때 => last_hidden_state 의 평균값으로 임베딩값구함 
    #===================================================================
    if onnxmodel: 
        # onnx 모델일때
        input_token=tokenizer(input, truncation=True, padding=True, max_length=256, return_tensors="pt")
        #print(input_token)
        outputs = model(**input_token)
        last_hidden_state = outputs.last_hidden_state
        #print(last_hidden_state.shape)

        vectors = []
        for hidden in last_hidden_state:
            # 평균값 구함
            mean_hidden = torch.mean(hidden, dim=0)
            vectors.append(mean_hidden)
        #return vectors
    #===================================================================
    # sentenctbert 모델일때
    #===================================================================
    else:
        vectors =  embedder.encode(input, convert_to_tensor=True)
    
    return [vector.numpy().tolist() for vector in vectors] 
          

In [7]:
#==============================================================================================
# 4. ES indices(index) 데이터 추가
# => 추가할 데이터는 {'paragraph': 내용, 'title': 제목} 기존 입려된 방식대로(사전) 입력 되어야 함
#===============================================================================================

# ES에 이미 생성된 index
INDEX_NAME = 'korquadonnx'
BATCH_SIZE = 30


# 1.추가할 데이터 준비
titles = [
    '제주도', 
    '한라산',
    '서울특별시',
    '파리',
    '로마',
    '프랑스',
    '마이크로소프트',
    '캥거루',
    '호랑이',
    '애플',
    '뉴턴',
    '레오나르도 다빈치',
    '기린',
    '해바라기',
    '튤립',
        ]

paragraphs = [
    '대한민국의 남서쪽에 있는 섬. 행정구역상 광역자치단체인 제주특별자치도의 관할. 한국의 섬 중에서 가장 크고 인구가 많은 섬으로 면적은 1833.2㎢이다. 제주도 다음 2번째 큰 섬인 거제도의 5배 정도 된다. 인구는 약 68만 명.',
    '대한민국에서 가장 큰 섬인 제주도에 있으며 대한민국의 실효지배 영토 내의 최고봉이자 가장 높은 산(해발 1,947m). 대한민국의 국립공원 중 하나이다. 국립공원 전역이 유네스코 세계유산으로 지정되었다.',
    '대한민국의 수도인 서울은 현대적인 고층 빌딩, 첨단 기술의 지하철, 대중문화와 예것이 공존하는 대도시. 주목할 만한 명소로는 초현대적 디자인의 컨벤션 홀인 동대문디자인플라자, 한때 7,000여 칸의 방이 자리하던 경복궁, 조계사가 있다',
    '파리는 프랑스의 수도이자 최대도시이다. 파리는 세계적으로도 문화, 정치, 외교, 경제의 큰 영향력을 끼치는 대도시이며, 특히 파리는 금융업이 발달해 있다',
    '로마는 이탈리아의 수도이자 최대 도시로, 라치오주의 주도이며, 테베레 강 연안에 있다.',
    '프랑스 공화국 약칭 프랑스는 서유럽의 본토와 남아메리카의 프랑스령 기아나를 비롯 대륙에 걸쳐 있는 해외 레지옹과 해외 영토 국가로서 EU 소속 국가 중 영토가 가장 넓다. 수도는 파리이다.',
    '컴퓨팅 파워를 지원해주는 클라우드 컴퓨팅 사업을 중심으로, 기업들을 지원하는 파워포인트, 워드와 엑셀, 원노트, 아웃룩, 팀즈 등의 오피스 365, Xbox 게임, 컴퓨터 운영체제 소프트웨어인 윈도우 등의 사업을 하는 미국의 기업이다.',
    '유대류 캥거루과에 속하는 초식동물의 총칭. 호주 대륙 본토 및 호주 북쪽의 뉴기니 섬에서 서식한다.',
    '호랑이 또는 범, 칡범, 갈범은 식육목 고양이과에 속하는 맹수다. 어린 개체는 개호주라 부른다. 고양이과 동물 중 그 크기가 가장 크다',
    '애플은 미국 캘리포니아의 아이폰, 아이패드, 애플 워치, 에어팟, 아이맥, 맥북, 맥 스튜디오와 맥 프로, 홈팟 등의 하드웨어와 iOS, iPadOS, macOS 등의 소프트웨어를 설계, 디자인하는 기업이다.',
    '아이작 뉴턴 경은 잉글랜드의 수학자, 물리학자, 천문학자이다. 1687년 발간된 자연철학의 수학적 원리는 고전역학과 만유인력의 기본 바탕을 제시하며, 과학사에서 영향력 있는 저서 중의 하나로 꼽힌다',
    '레오나르도 디 세르 피에로 다 빈치는 이탈리아 르네상스를 대표하는 석학이다. 화가이자 조각가, 발명가, 건축가, 해부학자, 지리학자, 음악가였다.',
    '기린은 기린과의 포유동물로서 기린속에 속하는 동물의 총칭이다. 목이 긴 육상 동물이며 또한 가장 큰 반추 동물이기도 하다.',
    '해바라기는 중앙아메리카가 원산지인 한해살이풀로, 해를 닮은 노란 꽃이 상당히 인상적인 식물이다.',
    '튤립은 백합과의 여러해살이풀로 산자고속 식물의 총칭이다. 울금향이라고도 한다. 남동 유럽과 중앙아시아가 원산지이다.',
            ]

# {'paragraph': "", 'title': ""}


doc = {}
docs = []
count = 0

# 3. batch 사이즈 만큼식 ES에 추가
# => 추가할 데이터는 {'paragraph': 내용, 'title': 제목} 기존 입려된 방식대로(사전) 입력 되어야 함
for title, paragraph in zip(titles, paragraphs):
    doc = {}
    doc['paragraph'] = paragraph
    doc['title'] = title
    docs.append(doc)
    count += 1
    if count % BATCH_SIZE == 0:
        index_batch(docs)
        docs = []
        print("Indexed {} documents.".format(count))
   
# docs 이 있으면 전송
if docs:
    index_batch(docs)
    print("Indexed {} documents(end).".format(count))


  0%|          | 0/15 [00:00<?, ?it/s]

Indexed 15 documents(end).


In [8]:

import time
from elasticsearch import Elasticsearch

def run_query_loop():
    while True:
        try:
            handle_query()
        except KeyboardInterrupt:
            return
        
def handle_query():
    
    query = input("검색어 입력: ")
    
    start_embedding_time = time.time()
    query_vector = embed_text([query])[0]
    end_embedding_time = time.time() - start_embedding_time
    
    # 쿼리 구성
    script_query = {
        "script_score":{
            "query":{
                "match_all": {}},
            "script":{
                "source": "cosineSimilarity(params.query_vector, doc['paragraph_vector']) + 1.0",  # 뒤에 1.0 은 코사인유사도 측정된 값 + 1.0을 더해준 출력이 나옴
                "params": {"query_vector": query_vector}
            }
        }
    }
    
    #print('query\n')
    #print(script_query)
    
    # 실제 ES로 검색 쿼리 날림
    start_search_time = time.time()
    response = es.search(
        index=INDEX_NAME,
        body={
            "size": SEARCH_SIZE,
            "query": script_query,
            "_source":{"includes": ["title", "paragraph"]}
        }
    )
    end_search_time = time.time() - start_search_time
    
    print("{} total hits.".format(response["hits"]["total"]["value"])) 
    print("embedding time: {:.2f} ms".format(end_embedding_time * 1000)) 
    print("search time: {:.2f} ms".format(end_search_time * 1000)) 
    print('\n')
    
    # 쿼리 응답 결과값에서 _id, _score, _source 등을 뽑아냄
    # print(response)
    
    for hit in response["hits"]["hits"]: 
        
        print("index:{}, type:{}".format(hit["_index"], hit["_type"]))
        print("id: {}, score: {}".format(hit["_id"], hit["_score"])) 
        
        print(f'[제목] {hit["_source"]["title"]}')
        
        print('[내용]')
        print(hit["_source"]["paragraph"]) 
        
        print()
    

In [12]:
#==============================================================================================
# 5. ES 인덱싱된 내용 검색 
# => cosineSimilarity 스크립트를 이용하여 ES로 query 함(*이때 SEARCH_SIZE를 몇개로 할지 지정할수 있음)
# => 쿼리 응답 결과 값에서 _id, _score, _source 등을 뽑아냄
#==============================================================================================
INDEX_NAME = 'korquadonnx'
SEARCH_SIZE = 2

# 2. query 처리
run_query_loop()

검색어 입력:  가장 가보고 싶은 곳은?


15 total hits.
embedding time: 8.00 ms
search time: 29.99 ms


index:korquadonnx, type:_doc
id: P15yA4EBWwNQ7PQ9mihl, score: 1.2043557
[제목] 한라산
[내용]
대한민국에서 가장 큰 섬인 제주도에 있으며 대한민국의 실효지배 영토 내의 최고봉이자 가장 높은 산(해발 1,947m). 대한민국의 국립공원 중 하나이다. 국립공원 전역이 유네스코 세계유산으로 지정되었다.

index:korquadonnx, type:_doc
id: Ql5yA4EBWwNQ7PQ9mihl, score: 1.1954284
[제목] 로마
[내용]
로마는 이탈리아의 수도이자 최대 도시로, 라치오주의 주도이며, 테베레 강 연안에 있다.



검색어 입력:  너무나 예쁜 것들


15 total hits.
embedding time: 6.00 ms
search time: 26.99 ms


index:korquadonnx, type:_doc
id: S15yA4EBWwNQ7PQ9mihl, score: 1.2000068
[제목] 해바라기
[내용]
해바라기는 중앙아메리카가 원산지인 한해살이풀로, 해를 닮은 노란 꽃이 상당히 인상적인 식물이다.

index:korquadonnx, type:_doc
id: QF5yA4EBWwNQ7PQ9mihl, score: 1.1165975
[제목] 서울특별시
[내용]
대한민국의 수도인 서울은 현대적인 고층 빌딩, 첨단 기술의 지하철, 대중문화와 예것이 공존하는 대도시. 주목할 만한 명소로는 초현대적 디자인의 컨벤션 홀인 동대문디자인플라자, 한때 7,000여 칸의 방이 자리하던 경복궁, 조계사가 있다



검색어 입력:  무서운 것들


15 total hits.
embedding time: 5.98 ms
search time: 29.99 ms


index:korquadonnx, type:_doc
id: Rl5yA4EBWwNQ7PQ9mihl, score: 1.1253055
[제목] 호랑이
[내용]
호랑이 또는 범, 칡범, 갈범은 식육목 고양이과에 속하는 맹수다. 어린 개체는 개호주라 부른다. 고양이과 동물 중 그 크기가 가장 크다

index:korquadonnx, type:_doc
id: RV5yA4EBWwNQ7PQ9mihl, score: 1.0958793
[제목] 캥거루
[내용]
유대류 캥거루과에 속하는 초식동물의 총칭. 호주 대륙 본토 및 호주 북쪽의 뉴기니 섬에서 서식한다.

